In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

/opt/miniconda3/envs/deploy_example/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_path = "/Users/syakubson/Desktop/PROJECTS/deploy_example/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    dtype="auto",
    device_map="auto"
)

In [4]:
# prepare the model input
prompt = "Tell me about bees"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True  # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=32768
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: <think>
Okay, the user asked about bees. Let me start by explaining what bees are. They are insects, right? I should mention that they belong to the hive, which is a group of insects. I need to explain their role in the ecosystem, like pollination.

I should cover their appearance: size, wings, and what they look like. Maybe mention their coloration and how they move. Then talk about their behavior—hiving, communication, and social structure. 

I should explain the hive structure, the queen, workers, and drones. Also, their importance in pollination and how they help with biodiversity. Make sure to keep it simple and avoid technical terms. Check if the user needs more info on a specific aspect, like honey production or their impact on humans. Keep the tone friendly and informative.
</think>
content: Bees are fascinating insects that play a crucial role in the ecosystem. Here's a brief overview:

### What Are Bees?
Bees are insects, but they are known for their social 